In [4]:
import requests
import pyinputplus as pyip
from pprint import pprint
from requests import Response
from pydantic import BaseModel,RootModel,Field,ValidationError,field_validator
from random import choices

class UB(BaseModel):
    場站中文名稱:str = Field(alias='sna')
    場站區域:str = Field(alias='sarea')
    資料更新時間:str = Field(alias='mday')
    地點:str = Field(alias='ar')
    act:bool
    場站總停車格:int = Field(alias='total')
    場站目前車輛數量:int = Field(alias='available_rent_bikes')
    空位數量:int = Field(alias='available_return_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')

class Root(RootModel):
    root:list[UB]

def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res:Response = requests.get(youbike_url)
    all_data = res.json()

    try:
        root:Root = Root.model_validate(all_data)
    except ValidationError as error:
        print(error)

    data_list:list[dict] = root.model_dump()
    filter_list:list[dict] = list(filter(lambda item:False if item['act']==0 else True,data_list))
    filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['場站目前車輛數量']<3 else False,data_list))
    filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))

    print(f"台北市YouBike有{len(data_list)}站點，目前正常運作站點共{len(filter_list)}個")

    num = pyip.inputInt("請輸入隨機顯示的站點數(1~5): ", min=1, max=5)
    print(num)

    print(f"\n可借車輛數為3輛以內的站點有{len(filter_list_rent_less3)}站")
    num_list:list[dict] = choices(filter_list_rent_less3, k=num)
    for sta in num_list:
            seq = filter_list_rent_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

    print(f"\n可還車輛數為3輛以內的站點有{len(filter_list_return_less3)}站")
    num_list:list[dict] = choices(filter_list_return_less3, k=num)
    for sta in num_list:
            seq = filter_list_return_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

if __name__ == '__main__':
     main()

台北市YouBike有1420站點，目前正常運作站點共1411個
請輸入隨機顯示的站點數(1~5): 2

可借車輛數為3輛以內的站點有383站
No.123
{ '場站中文名稱': 'YouBike2.0_景興路102巷口',
  '場站區域': '文山區',
  '資料更新時間': '2024-06-13 08:28:14',
  '地點': '景華街52巷/景興路102巷',
  'act': True,
  '場站總停車格': 14,
  '場站目前車輛數量': 0,
  '空位數量': 14,
  '緯度': 24.99438,
  '經度': 121.54376}
No.326
{ '場站中文名稱': 'YouBike2.0_虎林永吉路口',
  '場站區域': '信義區',
  '資料更新時間': '2024-06-13 09:21:22',
  '地點': '永吉路356號東側',
  'act': True,
  '場站總停車格': 15,
  '場站目前車輛數量': 1,
  '空位數量': 13,
  '緯度': 25.04519,
  '經度': 121.57712}

可還車輛數為3輛以內的站點有156站
No.3
{ '場站中文名稱': 'YouBike2.0_臺大男七舍前',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 09:19:19',
  '地點': '長興街/長興街80巷口西側',
  'act': True,
  '場站總停車格': 20,
  '場站目前車輛數量': 18,
  '空位數量': 2,
  '緯度': 25.01537,
  '經度': 121.54698}
No.93
{ '場站中文名稱': 'YouBike2.0_康寧路三段99巷17弄口',
  '場站區域': '內湖區',
  '資料更新時間': '2024-06-13 09:09:20',
  '地點': '康寧路三段99巷19號對側',
  'act': True,
  '場站總停車格': 15,
  '場站目前車輛數量': 13,
  '空位數量': 2,
  '緯度': 25.0721,
  '經度': 121.60991}
